Compute an optimal crossover

In [ ]:
import math
import time
import bisect

import pandas as pd
import numpy as np
import plotly as plt
import ray

pd.options.plotting.backend = "plotly"

import sys, os, os.path

sys.path.append(os.path.expanduser("../src"))

from generate_common import custom_ray_init, cache_load
from spinorama.filter_iir import Biquad
from spinorama.filter_peq import (
    peq_print,
    peq_format_apo,
    peq_spl,
    peq_linkwitzriley_lowpass,
    peq_linkwitzriley_highpass,
)
from spinorama.compute_cea2034 import compute_cea2034, estimated_inroom_hv
from spinorama.plot import plot_spinorama, common_layout, plot_graph_spl
from spinorama.load import shift_spl
import scipy.optimize as opt

In [ ]:
custom_ray_init({"--log-level": "INFO"})

In [ ]:
# speaker_name_lf = "Dolby CS 136LF"
# speaker_name_mh = "Dolby CS 136MH"
speaker_name_lf = "Dolby CS 128LF"
# speaker_name_mh = "Dolby CS 128MH"
speaker_name_mh = "Dolby CS 126MH"
speaker_origin = "Vendors-Dolby"
speaker_version = "vendor"
lf_speaker = cache_load({"speaker_name": speaker_name_lf, "origin": speaker_origin}, False)
mh_speaker = cache_load({"speaker_name": speaker_name_mh, "origin": speaker_origin}, False)

ray.shutdown()

In [ ]:
crossover_freq = 335
crossover_order_lp = 8
crossover_order_hp = 8

In [ ]:
spl_h_lf = lf_speaker[speaker_name_lf][speaker_origin][speaker_version]["SPL Horizontal_unmelted"]
spl_v_lf = lf_speaker[speaker_name_lf][speaker_origin][speaker_version]["SPL Vertical_unmelted"]

on_lf_mean = (
    spl_h_lf["On Axis"].loc[(spl_h_lf.Freq > 100) & (spl_h_lf.Freq < crossover_freq)].mean()
)

# normalise except Freq
spl_h_lf = shift_spl(spl_h_lf, on_lf_mean)
spl_v_lf = shift_spl(spl_v_lf, on_lf_mean)

spl_h_mh = mh_speaker[speaker_name_mh][speaker_origin][speaker_version]["SPL Horizontal_unmelted"]
spl_v_mh = mh_speaker[speaker_name_mh][speaker_origin][speaker_version]["SPL Vertical_unmelted"]

lf_spin = compute_cea2034(spl_h_lf, spl_v_lf)
lf_pir = estimated_inroom_hv(spl_h_lf, spl_v_lf)

mh_spin = compute_cea2034(spl_h_mh, spl_v_mh)
mh_pir = estimated_inroom_hv(spl_h_mh, spl_v_mh)

In [ ]:
plot_params_local = {
    "xmin": 20,
    "xmax": 20000,
    "ymin": -40,
    "ymax": 10,
    "width": 800,
    "height": 500,
}
plot_layout = common_layout(plot_params_local)
mh_plot = plot_spinorama(mh_spin, plot_params_local)
mh_plot.update_layout(
    {
        "legend": dict(
            x=1.4,
            y=1,
            xanchor="right",
            yanchor="top",
            orientation="v",
        ),
    }
)
mh_plot.show()
lf_plot = plot_spinorama(lf_spin, plot_params_local)
lf_plot.update_layout(
    {
        "legend": dict(
            x=1.4,
            y=1,
            xanchor="right",
            yanchor="top",
            orientation="v",
        ),
    }
)
lf_plot.show()

In [ ]:
freq = spl_h_lf.Freq
on_lf = spl_h_lf["On Axis"]
on_mh = spl_h_mh["On Axis"]

In [ ]:
lr_lp = peq_linkwitzriley_lowpass(crossover_order_lp, crossover_freq, 48000)
lr_hp = peq_linkwitzriley_highpass(crossover_order_hp, crossover_freq, 48000)

In [ ]:
# spl_lp = peq_build(freq, lr_lp)
# spl_hp = peq_build(freq, lr_hp)
# I need highshelves lowshelves not low/high pass (reverse of a crossover)
# let's use nothing for now
spl_lp = -100
spl_hp = -100

In [ ]:
def spl2pressure(spl):
    return np.power(10.0, np.divide(np.subtract(spl, 105.0), 20.0))


def pressure2spl(pressure):
    return np.add(105.0, np.multiply(20.0, np.log10(pressure)))


def spl_add(spl_a, spl_b):
    return pressure2spl(np.add(spl2pressure(spl_a), spl2pressure(spl_b)))

In [ ]:
spl_mh = spl_add(on_mh, spl_lp)
spl_lf = spl_add(on_lf, spl_hp)

fig = pd.DataFrame(
    {
        "Freq": freq,
        "lp": spl_lp,
        "hp": spl_hp,
        "mh+hp": spl_mh,
        "lf+lp": spl_lf,
        "mh": on_mh,
        "lf": on_lf,
    }
).plot.line(x="Freq", y=["lp", "hp", "mh", "lf", "mh+hp", "lf+lp"])
fig.update_xaxes(type="log")
fig.update_yaxes(type="linear", range=[-10, 10])
fig.update_layout(plot_layout)
fig.show()

In [ ]:
on = spl_add(spl_mh, spl_lf)

fig = pd.DataFrame(
    {
        "Freq": freq,
        "mh": spl_mh,
        "lf": spl_lf,
        "on": on,
    }
).plot.line(x="Freq", y=["on", "mh", "lf"])
fig.update_xaxes(type="log")
fig.update_yaxes(type="linear", range=[-10, 10])
fig.update_layout(plot_layout)
fig.show()

In [ ]:
def apply_crossover(spls, spl_crossover):
    df = pd.DataFrame()
    for key in spls.keys():
        if key == "Freq":
            df[key] = spls[key]
            continue
        df[key] = spl_add(spls[key], spl_crossover)
    return df


def merge_spl(spl_a, spl_b):
    df = pd.DataFrame()
    for key in spl_a.keys():
        if key == "Freq":
            df[key] = spl_a[key]
            continue
        df[key] = spl_add(spl_a[key], spl_b[key])
    return df


def merge_measurements(spl_h_lf, spl_v_lf, spl_lp, spl_h_mh, spl_v_mh, spl_hp):
    # lf
    h_lf = apply_crossover(spl_h_lf, spl_lp)
    v_lf = apply_crossover(spl_v_lf, spl_lp)
    # mh
    h_mh = apply_crossover(spl_h_mh, spl_hp)
    v_mh = apply_crossover(spl_v_mh, spl_hp)
    # sum
    return merge_spl(h_lf, h_mh), merge_spl(v_lf, v_mh)

In [ ]:
spl_h, spl_v = merge_measurements(spl_h_lf, spl_v_lf, spl_hp, spl_h_mh, spl_v_mh, spl_lp)

In [ ]:
spin = compute_cea2034(spl_h, spl_v)

In [ ]:
spin_plot = plot_spinorama(spin, plot_params_local)
spin_plot.update_layout(
    {
        "legend": dict(
            x=1.4,
            y=1,
            xanchor="right",
            yanchor="top",
            orientation="v",
        ),
    }
)
spin_plot.show()

spl_h_plot = plot_graph_spl(spl_h, plot_params_local)
spl_h_plot.update_layout(
    {
        "legend": dict(
            x=1.4,
            y=1,
            xanchor="right",
            yanchor="top",
            orientation="v",
        ),
    }
)
spl_h_plot.show()

spl_v_plot = plot_graph_spl(spl_v, plot_params_local)
spl_v_plot.update_layout(
    {
        "legend": dict(
            x=1.4,
            y=1,
            xanchor="right",
            yanchor="top",
            orientation="v",
        ),
    }
)
spl_v_plot.show()

In [ ]:
# speaker_name = "Dolby CS 133"
# speaker_name = "Dolby CS 128"
speaker_name = "Dolby CS 126"


def spl2files(spls, speaker_name: str, orientation: str):
    header = "Freq[Hz]     dBSPL  Phase[Deg]\n"
    freq = spls.Freq
    for spl in spls:
        angle = 0
        if spl == "Freq":
            continue
        if spl == "On Axis":
            angle = 0
        else:
            angle = int(spl[:-1])
        filename = f"{speaker_name} {orientation} {angle}.txt"
        with open(filename, "w", encoding="utf8") as fd:
            fd.writelines([header])
            fd.writelines(
                [
                    "{} {} {}\n".format(f, s, 0.0)
                    for f, s in zip(freq, spls[spl].values, strict=False)
                ]
            )


spl2files(spl_h, speaker_name, "_H")
spl2files(spl_v, speaker_name, "_V")